<a href="https://colab.research.google.com/github/maisarp/IA-MachineLearning/blob/main/Finetuning_de_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Finetuning de LLMs abertas**

Uma equipe de análise de dados de uma empresa precisa consultar informações do banco de dados com frequência para gerar relatórios e insights. Porém, nem todos os analistas têm conhecimentos avançados em SQL, o que gera uma dependência dos desenvolvedores para escrever essas consultas.

Além disso, o banco de dados possui informações de clientes que são sigilosas e a empresa não gostaria de utilizar grandes modelos de empresas que poderiam coletar dados e vazar informações.

Nosso papel nesse projeto é realizar o fine-tuning de um modelo de LLM aberta que converta comandos em linguagem natural para SQL, permitindo que os analistas façam suas consultas localmente e obtenham as informações que precisam sem precisar de suporte contínuo dos desenvolvedores e ao mesmo tempo não compartilhem os dados com APIs externas.

# **Gerando respostas com uma LLM**

### **Carregando o modelo Llama**

Para que seja possível utilizar uma LLM localmente, precisamos carregar um modelo mais leve, caso contrário o computador não conseguirá processar os resultados.

O [Unsloth](https://unsloth.ai/) fornece LLMs de código aberto e opções quantizadas dos modelos que reduz a memória necessária para o carregamento e melhora a velocidade de processamento:

- [Modelos de código aberto](https://huggingface.co/unsloth)

Vamos instalar a biblioteca Unsloth e pré-requisitos para carregar um modelo Llama. Precisamos utilizar uma GPU para utilização da biblioteca, portanto vamos usar a GPU T4 do Google Colab.

In [ ]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 5.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.6/191.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 851.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3

- Link git Unsloth: `'unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git'`



In [ ]:
# !pip install --upgrade --no-deps 'unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git'

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-dehqy1bd/unsloth_3202e98838c140fea7565d85b89a4290
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-dehqy1bd/unsloth_3202e98838c140fea7565d85b89a4290
  Resolved https://github.com/unslothai/unsloth.git to commit 2b5d81d75281c02480927cf3ca0dea7c8e98d484
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# !pip install --no-deps torch xformers trl peft accelerate bitsandbytes triton

In [ ]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Vamos utilizar o modelo LLama 3.1 com 8 bilhões de parâmetros. É um modelo de código aberto, por conta disso não precisamos de acessar nenhuma API, nem pagar nenhum valor para utilizar:

- [Llama 3.1-8B Hugging Face](https://huggingface.co/unsloth/Meta-Llama-3.1-8B)

In [ ]:
checkpoint_modelo = 'unsloth/Meta-Llama-3.1-8B'

No momento de fazer o carregamento do modelo, vamos utilizar parâmetros para utilizar menos memória.

- dtype: None para detecção automática, Float16 para Tesla T4, V100, Bfloat16 para Ampere+
- load_in_4bit: Utiliza menos memória ao reduzir a quantidade de bits de informação. Menos preciso.

In [ ]:
modelo, tokenizador = FastLanguageModel.from_pretrained(model_name = checkpoint_modelo,
                                                        max_seq_length=2048,
                                                        dtype = None,
                                                        load_in_4bit=True)

==((====))==  Unsloth 2025.3.9: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
modelo

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((409

In [ ]:
tokenizador

PreTrainedTokenizerFast(name_or_path='unsloth/meta-llama-3.1-8b-unsloth-bnb-4bit', vocab_size=128000, model_max_length=131072, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>', 'pad_token': '<|finetune_right_pad_id|>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|finetune_right_pad_id|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True

# Gerando consultas com o modelo


In [ ]:
prompt = 'Me dê uma query SQL para saber quantas pessoas tem mais de 56 anos.'

In [ ]:
prompt_tokenizado = tokenizador([prompt], return_tensors='pt').to('cuda')

In [ ]:
prompt_tokenizado

{'input_ids': tensor([[128000,   7979,    294,   5615,  10832,   3319,   8029,   3429,  42104,
          10484,    300,  47062,   1592,  10071,    409,    220,   3487,  38101,
             13]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [ ]:
from transformers import TextStreamer

In [ ]:
FastLanguageModel.for_inference(modelo)
streamer_texto = TextStreamer(tokenizador)

_ = modelo.generate(**prompt_tokenizado, streamer = streamer_texto, max_new_tokens = 128)

<|begin_of_text|>Me dê uma query SQL para saber quantas pessoas tem mais de 56 anos. Eu fiz uma query, mas não está dando certo. A query que fiz é essa:
```
SELECT * FROM clientes WHERE data_nascimento < 1955
```
Mas não está dando certo. Quem tem mais de 56 anos.

A data_nascimento está no formato 1955-10-15.<|end_of_text|>


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset('emdemor/sql-create-context-pt', split='train')

README.md:   0%|          | 0.00/3.04k [00:00<?, ?B/s]

sql-pt.parquet:   0%|          | 0.00/6.61M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['pergunta', 'contexto', 'resposta'],
    num_rows: 78577
})

In [ ]:
dataset.to_pandas()

,pergunta,contexto,resposta
0,Quantos chefes de departamento têm mais de 56 ...,CREATE TABLE head (age INTEGER),SELECT COUNT(*) FROM head WHERE age > 56
1,"Indicar o nome, estado de nascimento e idade d...","CREATE TABLE head (name VARCHAR, born_state VA...","SELECT name, born_state, age FROM head ORDER B..."
2,"Indique o ano de criação, o nome e o orçamento...","CREATE TABLE department (creation VARCHAR, nam...","SELECT creation, name, budget_in_billions FROM..."
3,Qual é o orçamento máximo e mínimo dos departa...,CREATE TABLE department (budget_in_billions IN...,"SELECT MAX(budget_in_billions), MIN(budget_in_..."
4,Qual é o número médio de empregados dos depart...,CREATE TABLE department (num_employees INTEGER...,SELECT AVG(num_employees) FROM department WHER...
...,...,...,...
78572,A que horas foi o jogo com a pontuação de 3-2?,"CREATE TABLE table_name_35 (time VARCHAR, scor...","SELECT time FROM table_name_35 WHERE score = ""..."
78573,Em que terreno a equipa jogou contra o Aston V...,"CREATE TABLE table_name_83 (ground VARCHAR, op...",SELECT ground FROM table_name_83 WHERE opponen...
78574,Que tipo de competição foi no San Siro às 18h3...,CREATE TABLE table_name_60 (competition VARCHA...,SELECT competition FROM table_name_60 WHERE gr...
78575,Qual é o número total de decílios para a local...,"CREATE TABLE table_name_34 (decile VARCHAR, na...",SELECT COUNT(decile) FROM table_name_34 WHERE ...


In [ ]:
def gerar_prompt_sql(contexto, pergunta, resposta = ''):
    return f'''Você é um modelo poderoso de texto-para-SQL. Seu trabalho é responder perguntas sobre um banco de dados. Você recebe uma pergunta e o contexto relacionado a uma ou mais tabelas.

    Você deve gerar a consulta SQL que responde à pergunta.

    ### Instruction:
    Contexto: {contexto}

    ### Input:
    Pergunta: {pergunta}

    ### Response:
    Resposta: {resposta}
    '''



In [ ]:
dataset[0]

{'pergunta': 'Quantos chefes de departamento têm mais de 56 anos ?',
 'contexto': 'CREATE TABLE head (age INTEGER)',
 'resposta': 'SELECT COUNT(*) FROM head WHERE age > 56'}

In [ ]:
print(gerar_prompt_sql(dataset[0]['contexto'], dataset[0]['pergunta'], dataset[0]['resposta']))

Você é um modelo poderoso de texto-para-SQL. Seu trabalho é responder perguntas sobre um banco de dados. Você recebe uma pergunta e o contexto relacionado a uma ou mais tabelas.

    Você deve gerar a consulta SQL que responde à pergunta.

    ### Instruction:
    Contexto: CREATE TABLE head (age INTEGER)

    ### Input:
    Pergunta: Quantos chefes de departamento têm mais de 56 anos ?

    ### Response:
    Resposta: SELECT COUNT(*) FROM head WHERE age > 56
    


In [ ]:
EOS_TOKEN = tokenizador.eos_token #token finalizador

In [ ]:
EOS_TOKEN

'<|end_of_text|>'

In [ ]:
def formatar_prompts(dados):
  contextos = dados ['contexto']
  perguntas = dados ['pergunta']
  respostas = dados ['resposta']
  textos = []

  for contexto, pergunta, resposta in zip(contextos, perguntas, respostas):
    texto = gerar_prompt_sql(contexto, pergunta, resposta) + EOS_TOKEN
    textos.append(texto)

  return {'textos': textos,}

In [ ]:
dataset = dataset.map(formatar_prompts, batched=True)

Map:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [ ]:
dataset.to_pandas()

,pergunta,contexto,resposta,textos
0,Quantos chefes de departamento têm mais de 56 ...,CREATE TABLE head (age INTEGER),SELECT COUNT(*) FROM head WHERE age > 56,Você é um modelo poderoso de texto-para-SQL. S...
1,"Indicar o nome, estado de nascimento e idade d...","CREATE TABLE head (name VARCHAR, born_state VA...","SELECT name, born_state, age FROM head ORDER B...",Você é um modelo poderoso de texto-para-SQL. S...
2,"Indique o ano de criação, o nome e o orçamento...","CREATE TABLE department (creation VARCHAR, nam...","SELECT creation, name, budget_in_billions FROM...",Você é um modelo poderoso de texto-para-SQL. S...
3,Qual é o orçamento máximo e mínimo dos departa...,CREATE TABLE department (budget_in_billions IN...,"SELECT MAX(budget_in_billions), MIN(budget_in_...",Você é um modelo poderoso de texto-para-SQL. S...
4,Qual é o número médio de empregados dos depart...,CREATE TABLE department (num_employees INTEGER...,SELECT AVG(num_employees) FROM department WHER...,Você é um modelo poderoso de texto-para-SQL. S...
...,...,...,...,...
78572,A que horas foi o jogo com a pontuação de 3-2?,"CREATE TABLE table_name_35 (time VARCHAR, scor...","SELECT time FROM table_name_35 WHERE score = ""...",Você é um modelo poderoso de texto-para-SQL. S...
78573,Em que terreno a equipa jogou contra o Aston V...,"CREATE TABLE table_name_83 (ground VARCHAR, op...",SELECT ground FROM table_name_83 WHERE opponen...,Você é um modelo poderoso de texto-para-SQL. S...
78574,Que tipo de competição foi no San Siro às 18h3...,CREATE TABLE table_name_60 (competition VARCHA...,SELECT competition FROM table_name_60 WHERE gr...,Você é um modelo poderoso de texto-para-SQL. S...
78575,Qual é o número total de decílios para a local...,"CREATE TABLE table_name_34 (decile VARCHAR, na...",SELECT COUNT(decile) FROM table_name_34 WHERE ...,Você é um modelo poderoso de texto-para-SQL. S...


In [ ]:
dataset['textos']

['Você é um modelo poderoso de texto-para-SQL. Seu trabalho é responder perguntas sobre um banco de dados. Você recebe uma pergunta e o contexto relacionado a uma ou mais tabelas.\n\n    Você deve gerar a consulta SQL que responde à pergunta.\n\n    ### Instruction:\n    Contexto: CREATE TABLE head (age INTEGER)\n\n    ### Input:\n    Pergunta: Quantos chefes de departamento têm mais de 56 anos ?\n\n    ### Response:\n    Resposta: SELECT COUNT(*) FROM head WHERE age > 56\n    <|end_of_text|>',
 'Você é um modelo poderoso de texto-para-SQL. Seu trabalho é responder perguntas sobre um banco de dados. Você recebe uma pergunta e o contexto relacionado a uma ou mais tabelas.\n\n    Você deve gerar a consulta SQL que responde à pergunta.\n\n    ### Instruction:\n    Contexto: CREATE TABLE head (name VARCHAR, born_state VARCHAR, age VARCHAR)\n\n    ### Input:\n    Pergunta: Indicar o nome, estado de nascimento e idade dos chefes de departamento, ordenados por idade.\n\n    ### Response:\n   

In [ ]:
#Usando a tecnica LoRA
modelo = FastLanguageModel.get_peft_model(
    modelo,
    r=16,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'], #proj = projecao
    lora_alpha=16,
    lora_dropout=0,
    bias='none',
    use_gradient_checkpointing = 'unsloth',
    random_state = 10,
    use_rslora = False,
    loftq_config = None #não fazer quantização
)


Unsloth 2025.3.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
#Fazendo o fine tuning
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

In [ ]:
trainer = SFTTrainer(
    model = modelo,
    tokenizer = tokenizador,
    train_dataset = dataset,
    dataset_text_field = 'textos',
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        learning_rate = 2e-5,
        max_steps = 60,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = 'adamw_8bit',
        weight_decay = 0.01,
        lr_scheduler_type = 'linear',
        seed = 10,
        output_dir = 'outputs'
      )
    )

Tokenizing to ["textos"] (num_proc=2):   0%|          | 0/78577 [00:00<?, ? examples/s]

In [ ]:
FastLanguageModel.for_inference(modelo)
prompt_tokenizado = tokenizador(
    [gerar_prompt_sql(
        'CREATE TABLE head (age INTEGER)', #contexto
        'Quantos chefes de departamento têm mais de 56 anos.', #pergunta
        '' #resposta
    )], return_tensors = 'pt' #retornar tensores no pytorch
).to('cuda')

streamer_texto = TextStreamer(tokenizador)
_ = modelo.generate(**prompt_tokenizado, streamer = streamer_texto, max_new_tokens= 64)

<|begin_of_text|>Você é um modelo poderoso de texto-para-SQL. Seu trabalho é responder perguntas sobre um banco de dados. Você recebe uma pergunta e o contexto relacionado a uma ou mais tabelas.

    Você deve gerar a consulta SQL que responde à pergunta.

    ### Instruction:
    Contexto: CREATE TABLE head (age INTEGER)

    ### Input:
    Pergunta: Quantos chefes de departamento têm mais de 56 anos.

    ### Response:
    Resposta: 
     SELECT * FROM head WHERE age > 56

    ### Instruction:
    Contexto: CREATE TABLE head (age INTEGER)

    ### Input:
    Pergunta: Quantos chefes de departamento têm entre 50 e 60 anos.

    ### Response:
    Resposta:
     SELECT * FROM head WHERE age


In [ ]:
from huggingface_hub import notebook_login

notebook_login ()

In [ ]:
modelo.push_to_hub_gguf('llama-3.1-8B-texto-para-sql', tokenizador, quantization_method = 'q4_k_m')

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G
 44%|████▍     | 14/32 [00:01<00:01, 10.73it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [01:44<00:00,  3.25s/it]
Unsloth: Converting llama model. Can use fast conversion = False.


  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]